<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

math.sqrt
https://docs.python.org/3/tutorial/floatingpoint.html
numpy
np.sqrt()
matplotlib.pyplot
https://tour.golang.org/flowcontrol/8 Newtons method
https://www.mathjax.org/ - built in to jupyter notebook (LaTeX math symbols)